In [1]:
import numpy
import torch
import mobster as mb

import numpy as np


In [2]:
data = mb.load_example_data("./")

In [3]:
import pyro

pyro.enable_validation(True)
fitted = mb.fit_mobster(data,1,1, subclonal_prior="Beta", multi_tail = False, purity = 0.9,  truncated_pareto=True, max_it = 1000, lr = 0.01,  e = 0.005, lrd_gamma = 0.1,
        number_of_trials_k= 1000, prior_lims_k=[1,1000000], prior_lims_clonal=[1,1000000.], number_of_trials_clonal_mean= 1000)


Running MOBSTER on 4 karyotypes with 1 subclones.
Fitting a model with tail


ELBO: 46671.487134339  :  58%|█████▊    | 584/1000 [01:07<00:48,  8.60it/s]

Computing cluster assignements.


Computing information criteria.
Done!



In [6]:


mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [4]:
fitted["model_parameters"]



{'1:0': {'cluster_probs': array([[4.1139135e-03, 2.7543779e-03, 2.9391225e-03, ..., 5.3838454e-03,
          1.0000000e+00, 3.7982892e-02],
         [9.6902275e-01, 9.7670627e-01, 9.7569078e-01, ..., 9.6167892e-01,
          3.0396763e-31, 7.9033858e-01],
         [2.6863437e-02, 2.0539468e-02, 2.1370202e-02, ..., 3.2937229e-02,
          2.1590541e-08, 1.7167842e-01]], dtype=float32),
  'cluster_assignments': array(['C1', 'C1', 'C1', ..., 'C1', 'Tail', 'C1'], dtype=object),
  'cluster_types': array(['Tail', 'C1', 'S1'], dtype='<U4'),
  'mixture_probs': array([0.02426884, 0.92924947, 0.04648174], dtype=float32),
  'beta_concentration1': array([1197.4833], dtype=float32),
  'beta_concentration2': array([354.43954], dtype=float32),
  'dispersion_noise': 0.050639689637180484,
  'ccf_subclones': array([0.77497447], dtype=float32),
  'loc_subclones': array([0.697477], dtype=float32),
  'n_trials_subclonal': array([30.826311], dtype=float32),
  'tail_shape': array([0.6377746], dtype=float32)